In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import string
import os
import pandas as pd
import datetime
import numpy as np
import statsmodels.api as sm
import math
from scipy.linalg import svd
#import Functions as Fun
import scipy.linalg as la
import scipy.sparse.linalg as sla

In [ ]:
X_all = pd.read_csv('../input/project2/X_cleaned.csv')
Y_all = pd.read_csv('../input/project2/Y_cleaned.csv')
Date_index = pd.read_csv('../input/project2/Date_list.csv')
Date_index['DATE']=pd.to_datetime(Date_index['DATE'])

In [ ]:
X_all[X_all.columns[0]]

In [ ]:
nums= 30
start_dates = [pd.to_datetime('1957-03-01')+pd.offsets.DateOffset(years=x) for x in range(nums) ]

In [ ]:
R2_oos=0
for m in range(len(start_dates)):

    start_date = start_dates[m]

    end_date =  start_date + pd.offsets.DateOffset(years=18)

    ind_training = (start_date < Date_index['DATE']) & (Date_index['DATE'] < end_date)

    ytrain = Y_all[ind_training]

    xtrain = X_all[ind_training]

    ####for validation

    start_date =  end_date
    end_date = start_date + pd.offsets.DateOffset(years=12)

    ind_vali = (start_date < Date_index['DATE']) & (Date_index['DATE'] < end_date)

    ytest = Y_all[ind_vali]
    xtest= X_all[ind_vali]

    #####for test  oos

    start_date =  end_date
    end_date = start_date + pd.offsets.DateOffset(years=1)
    ind_test = (start_date < Date_index['DATE']) & (Date_index['DATE'] < end_date)
    #print(type(ind_test))
    yoos = Y_all[ind_test]
    xoos= X_all[ind_test]

    #######Monthly Demean %%%
    ytrain_demean= ytrain - np.mean(ytrain);
    ytest_demean=ytest-np.mean(ytest);
    mtrain=np.mean(ytrain);
    mtest=np.mean(ytest);

    sd=np.zeros(len(xtrain.columns)) #dim of sd? sd for each characteristics
    for i in range(len(xtrain.columns)):
        s=np.std(xtrain.iloc[:,[i]])
        colnames = xtrain.columns
        if s.values > 0:
            colname = colnames[i]
            xtrain.loc[:,colname]=xtrain[colname]/s.values
            xtest.loc[:,colname]=xtest.loc[:,colname]/s.values
            xoos.loc[:,colname]=xoos.loc[:,colname]/s.values
            sd[i]=s.values
    #### Start to Train %%%
    #model fitting

    clf=sm.OLS(ytrain_demean,xtrain)
    yhatres = clf.fit()
    yhatbig1= yhatres.predict(xoos)+ mtrain.values
    #prediction for oos
    r2_oos=1-sum((yhatbig1-yoos['RET'])**2)/sum((yoos['RET']-mtrain.values)**2) # oos r2
    yhatbig1=yhatres.predict(xtrain) + mtrain.values
    r2_is=1-(yhatbig1-ytrain['RET']).pow(2).sum()/sum(pow(ytrain['RET']-mtrain.values,2))
    R2_oos=R2_oos+r2_oos
print('R2_oos:'+np.str0(R2_oos/30))

<!--
R2_oos:-1410.2865969719674

In [ ]:
for i in range(len(X_all.columns)):
    R2_oos_1=0
    for m in range(len(start_dates)):

        start_date = start_dates[m]

        end_date =  start_date + pd.offsets.DateOffset(years=18)

        ind_training = (start_date < Date_index['DATE']) & (Date_index['DATE'] < end_date)

        ytrain = Y_all[ind_training]

        xtrain = X_all[ind_training]

    ####for validation

        start_date =  end_date
        end_date = start_date + pd.offsets.DateOffset(years=12)

        ind_vali = (start_date < Date_index['DATE']) & (Date_index['DATE'] < end_date)

        ytest = Y_all[ind_vali]
        xtest= X_all[ind_vali]

    #####for test  oos

        start_date =  end_date
        end_date = start_date + pd.offsets.DateOffset(years=1)
        ind_test = (start_date < Date_index['DATE']) & (Date_index['DATE'] < end_date)
    #print(type(ind_test))
        yoos = Y_all[ind_test]
        xoos= X_all[ind_test]

    #######Monthly Demean %%%
        ytrain_demean= ytrain - np.mean(ytrain);
        ytest_demean=ytest-np.mean(ytest);
        mtrain=np.mean(ytrain);
        mtest=np.mean(ytest);

        sd=np.zeros(len(xtrain.columns)) #dim of sd? sd for each characteristics
        for j in range(len(xtrain.columns)):
            s=np.std(xtrain.iloc[:,[j]])
            colnames = xtrain.columns
            if s.values > 0:
                colname = colnames[j]
                xtrain.loc[:,colname]=xtrain[colname]/s.values
                xtest.loc[:,colname]=xtest.loc[:,colname]/s.values
                xoos.loc[:,colname]=xoos.loc[:,colname]/s.values
                sd[j]=s.values
                
        xtrain_1=xtrain.drop(columns=[X_all.columns[i]])
        xoos_1= xoos.drop(columns=[X_all.columns[i]])
        clf=sm.OLS(ytrain_demean,xtrain_1)
        yhatres = clf.fit()
        yhatbig1= yhatres.predict(xoos_1)+ mtrain.values
        #prediction for oos
        r2_oos_1=1-sum((yhatbig1-yoos['RET'])**2)/sum((yoos['RET']-mtrain.values)**2) # oos r2
        yhatbig1= yhatres.predict(xtrain_1)+ mtrain.values
        r2_is_1=1-(yhatbig1-ytrain['RET']).pow(2).sum()/sum(pow(ytrain['RET']-mtrain.values,2))
        R2_oos_1=R2_oos_1+r2_oos_1
    d= (R2_oos-R2_oos_1)/30 
    print(X_all.columns[i]+':'+np.str0(d))



<!--
beta:-1.50917918684839
betasq:0.5101543974240486
chmom:-10.285963619522954
dolvol:82.19699529798987
idiovol:0.15936835547496836
indmom:-8.21975190007215
mom1m:166.11982724354215
mom6m:-3.555373215624907
mom12m:-1.6712752451513855
mom36m:2.0419622718259536
pricedelay:-0.5982017299803071
turn:0.04414510546145418
absacc:-3.5166215226985513
acc:3.336899165554981
age:-12.508876248853145
agr:-0.03489651214137363
cashdebt:-1.303915277638104
cashpr:-4.557326076759879
cfp:-7.045873446459397
cfp_ia:2.832360661693868
chatoia:2.36531959690304
chcsho:1.0465218841757935
chempia:-0.21549158756267087
chinv:-2.317842227393703
chpmia:7.612786754267775
convind:-1.1625522306009468
currat:0.5125223758868136
depr:42.685893212691006
divi:-1.6898299990852441
divo:3.949812090020229
dy:-5.7696128058940905
egr:2.8176175459627606
ep:10.622469988885133
gma:15.95631341950284
grcapx:-5.8219799173850335
grltnoa:13.524548201261496
herf:0.8799715706576535
hire:1.4063356024465368
invest:-0.37801043730117573
lev:1.8538568367897823
lgr:-5.2653738454918
mve_ia:0.6695397095119309
operprof:-1.0553216985613594
orgcap:15.311161927467037
pchcapx_ia:-1.62685780779575
pchcurrat:-5.230648057286696
pchdepr:-4.607420166401183
pchgm_pchsale:-8.957687686506931
pchquick:-4.919130740488132
pchsale_pchinvt:0.47730918651723186
pchsale_pchrect:1.4866766034596366
pchsale_pchxsga:3.449856071211737
pchsaleinv:-0.22581172841722338
pctacc:3.2626620067070085
ps:-2.024208231107574
quick:0.033414641598210434
rd:-2.1070582613002142
rd_mve:5.81610883730973
rd_sale:-0.8792749568274303
realestate:-0.39097001625414124
roic:4.641404521740332
salecash:3.2261751257972113
saleinv:5.241790553811248
salerec:-0.9014437456273299
secured:-1.954097117651933
securedind:-20.86786565099416
sgr:-0.14320374051094403
sin:0.1683315395093814
sp:6.692859310945641
tang:6.492455090613173
tb:-1.6841308488340776
aeavol:1.0474785761136445
cash:-4.189245135881113
chtx:1.1293763367956369
cinvest:57.471181142140026
ear:-6.309188255400416
nincr:-2.8129590227746424
roaq:-25.091739610382987
roavol:47.75121561413131
roeq:11.743813406523502
rsup:-0.3749843692545255
stdacc:-1361.8759275900177
stdcf:-1363.0364594764098
ms:-1.0896403002542987
baspread:-4.850038392924519
ill:27.59290402188708
maxret:12.133797021455393
retvol:32.63039629633398
std_dolvol:15.270880235526905
std_turn:44.4297792573474
zerotrade:4.120298216848217
bm:5.524287447170597
bm_ia:9.43023219246752
sic2_10:-2.724192643930049
sic2_13:58.805430278853116
sic2_15:9.323407701539448
sic2_17:-1.1641532182693482e-11
sic2_20:64.35701231737887
sic2_22:46.2322639548358
sic2_23:36.419425424079236
sic2_24:39.171258078729686
sic2_25:2.7361935887408133e-05
sic2_26:74.84429857154245
sic2_27:39.555346609843156
sic2_28:74.51094970236615
sic2_29:39.759566120938445
sic2_30:12.750897094114043
sic2_32:22.942034450935655
sic2_33:60.37553932909311
sic2_34:53.259309780816935
sic2_35:70.76029466511973
sic2_36:56.73260535518275
sic2_37:54.275184013204125
sic2_38:51.31776345491453
sic2_39:53.51393607016459
sic2_40:49.303059651258806
sic2_41:-0.0001990531432966236
sic2_42:0.010922281356276169
sic2_44:28.498868324191182
sic2_45:24.055876853760857
sic2_47:-0.8437599860158419
sic2_48:-0.5931148969425218
sic2_49:69.75060776075527
sic2_50:44.80730518684844
sic2_51:-2.9437845571461367
sic2_52:0.07163806536167007
sic2_53:0.2309283966863101
sic2_54:51.19775096197409
sic2_56:-0.0010587738480050271
sic2_57:-1.3854683418821272
sic2_58:28.70327969551848
sic2_59:0.2727214786859501
sic2_60:25.128254115194675
sic2_61:1.8788357085012346
sic2_62:-2.6839378907519857
sic2_63:11.610542798797058
sic2_64:-0.011688424634242741
sic2_65:-1.1954446270351278
sic2_67:31.256093828405575
sic2_70:-4.136553131199131e-05
sic2_72:0.320957553544334
sic2_73:46.62637114007157
sic2_75:28.948178026157013
sic2_78:55.60120610675852
sic2_79:-3.963799374109173
sic2_80:-5.122992103547828
sic2_87:43.63111942094547
sic2_99:-0.009069994922416906